In [ ]:
# Author: HZQ, ZTR
# Last modified: 2019/4/3

In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools 
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null 
!apt-get update -qq 2>&1 > /dev/null 
!apt-get -y install -qq google-drive-ocamlfuse fuse 
from google.colab import auth 
auth.authenticate_user() 
from oauth2client.client import GoogleCredentials 
creds = GoogleCredentials.get_application_default() 
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL 
vcode = getpass.getpass() 
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [ ]:
!mkdir -p driver
!google-drive-ocamlfuse driver
import os

os.chdir("driver/")
!ls

In [ ]:
!pip install catboost
!pip install plotly

In [1]:
import gc
import os
import string
import time
import random
import warnings 
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 
# from plotly import tools
import plotly.offline as py
# import plotly.graph_objs as go

# from scipy.stats import norm
# from scipy import stats
# from sklearn.preprocessing import StandardScaler
# from sklearn import model_selection, preprocessing, metrics, ensemble, naive_bayes, linear_model
# from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import StratifiedKFold, KFold

import xgboost as xgb
import lightgbm as lgb
import catboost as cb


pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999
color = sns.color_palette()
%matplotlib inline
py.init_notebook_mode(connected=True)


data_root = '../../santander-customer-transaction-prediction'

In [2]:
train_df = pd.read_csv(os.path.join(data_root, 'train.csv'))
test_df = pd.read_csv(os.path.join(data_root, 'test.csv'))

feature_names = [c for c in train_df.columns if c not in ['ID_code', 'target']]
X = train_df[feature_names].values.astype(np.float32)
y = train_df['target'].values
T = test_df[feature_names].values.astype(np.float32)

In [3]:
# Taking a look at how many rows and columns the train dataset contains
rows1 = train_df.shape[0]; rows2 = test_df.shape[0]
columns1 = train_df.shape[1]; columns2 = test_df.shape[1]
print("The train dataset contains {0} rows and {1} columns".format(rows1, columns1))
print("The test dataset contains {0} rows and {1} columns".format(rows2, columns2))

The train dataset contains 200000 rows and 202 columns
The test dataset contains 200000 rows and 201 columns


In [4]:
"""
XGBoost model
"""
# https://www.kaggle.com/silverstone1903/xgboost-baseline
xgb_params = {'tree_method': 'hist',
              'objective': 'binary:logistic',
              'eval_metric': 'auc',
              'learning_rate': 0.0936165921314771,
              'max_depth': 2,
              'colsample_bytree': 0.3561271102144279,
              'subsample': 0.8246604621518232,
              'min_child_weight': 53,
              'gamma': 9.943467991283027,
              'silent': 1,
              'num_boost_round': 5000,
              'early_stopping_rounds': 500
             }

xgboost_model = xgb.XGBClassifier(**xgb_params)

In [5]:
"""
LightGBM model
"""
# https://www.kaggle.com/fayzur/customer-transaction-prediction-strong-baseline
# Thanks fayzur. Nice Parameter
lgb_params = {'bagging_freq': 5,
              'bagging_fraction': 0.4,
              'boost_from_average':'false',
              'boost': 'gbdt',
              'feature_fraction': 0.05,
              'learning_rate': 0.01,
              'max_depth': -1,  
              'metric':'auc',
              'min_data_in_leaf': 80,
              'min_sum_hessian_in_leaf': 10.0,
              'num_leaves': 13,
              'num_threads': 8,
              'tree_learner': 'serial',
              'objective': 'binary', 
              'verbosity': 1,
              'num_boost_round': 10000,
              'early_stopping_rounds': 500
             }

stacker_params = {'bagging_freq': 5,
                  'bagging_fraction': 0.4,
                  'boost_from_average':'false',
                  'boost': 'gbdt',
                  'feature_fraction': 0.05,
                  'learning_rate': 0.01,
                  'max_depth': -1,  
                  'metric':'auc',
                  'min_data_in_leaf': 80,
                  'min_sum_hessian_in_leaf': 10.0,
                  'num_leaves': 13,
                  'num_threads': 8,
                  'tree_learner': 'serial',
                  'objective': 'binary', 
                  'verbosity': 1,
                  'num_boost_round': 10000,
                 }

lightgbm_model = lgb.LGBMClassifier(**lgb_params)
lightgbm_stacker_model = lgb.LGBMClassifier(**stacker_params)

In [18]:
"""
CatBoost model
"""
# https://www.kaggle.com/wakamezake/starter-code-catboost-baseline
cb_params = {'loss_function': 'Logloss',
             'eval_metric': 'AUC',
             'learning_rate': 0.01,
             'iterations': 10000,
             'random_seed': 42,
             'od_type': 'Iter',
             'depth': 10,
             'early_stopping_rounds': 500
            }

catboost_model = cb.CatBoostClassifier(**cb_params)

In [24]:
def train_predict(name, classifier, X, y, T, n_folds=5):
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=2019)
    pred = np.zeros(T.shape[0])
    
    for i, (train_idx, test_idx) in enumerate(skf.split(X, y)):
        X_train = X[train_idx]
        y_train = y[train_idx]
        X_eval = X[test_idx]
        y_eval = y[test_idx]
        
        if name == 'catboost':
            data_train = cb.Pool(X_train, label=y_train)
            data_eval = cb.Pool(X_eval, label=y_eval)
            classifier.fit(data_train, eval_set=data_eval, use_best_model=True)
            pred += classifier.predict(T, prediction_type='Probability')[:, 1]
        else:
            data_eval = (X_eval, y_eval)
            classifier.fit(X_train, y_train, eval_set=data_eval)
            pred += classifier.predict_proba(T)[:, 1]
        
    pred /= n_folds
    
    return pred

In [20]:
class Ensemble(object):
    
    def __init__(self, stacker, base_models, model_names, n_folds=5):
        super().__init__()
        self.stacker = stacker
        self.base_models = base_models
        self.model_names = model_names
        self.n_folds = n_folds

    def fit_predict(self, X, y, T):
        X = np.array(X)
        y = np.array(y)
        T = np.array(T)
        
        skf = StratifiedKFold(n_splits=self.n_folds, shuffle=True, random_state=2019)

        S_train = np.zeros((X.shape[0], len(self.base_models)))
        S_test = np.zeros((T.shape[0], len(self.base_models)))

        for i, (clf, name) in enumerate(zip(self.base_models, self.model_names)):
            S_test_i = np.zeros((T.shape[0], self.n_folds))

            for j, (train_idx, test_idx) in enumerate(skf.split(X, y)):
                X_train = X[train_idx]
                y_train = y[train_idx]
                X_holdout = X[test_idx]
                y_holdout = y[test_idx]
                
                if name == 'catboost':
                    data_train = cb.Pool(X_train, label=y_train)
                    data_eval = cb.Pool(X_eval, label=y_eval)
                    clf.fit(data_train, eval_set=data_eval, use_best_model=True)
                    y_pred = clf.predict(X_holdout, prediction_type='Probability')[:, 1]
                else:
                    data_eval = (X_holdout, y_holdout)
                    clf.fit(X_train, y_train, eval_set=data_eval)
                    y_pred = clf.predict_proba(X_holdout)[:, 1]
                    
                S_train[test_idx, i] = y_pred
                S_test_i[:, j] = clf.predict(T)[:]

            S_test[:, i] = S_test_i.mean(1)

        self.stacker.fit(S_train, y)
        y_pred = self.stacker.predict_proba(S_test)[:, 1]
        
        return y_pred

In [ ]:
# XGBoost
name = 'xgboost'
pred = train_predict(name, xgboost_model, X, y, T, n_folds=5)

In [16]:
# LightGBM
name = 'lightgbm'
pred = train_predict(name, lightgbm_model, X, y, T, n_folds=5)

/home/ztr/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:118: UserWarning:

Found `num_boost_round` in params. Will use it instead of argument

/home/ztr/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:123: UserWarning:

Found `early_stopping_rounds` in params. Will use it instead of argument



[1]	valid_0's auc: 0.590364
Training until validation scores don't improve for 500 rounds.
[2]	valid_0's auc: 0.634246
[3]	valid_0's auc: 0.66408
[4]	valid_0's auc: 0.671075
[5]	valid_0's auc: 0.686863
[6]	valid_0's auc: 0.705956
[7]	valid_0's auc: 0.713168
[8]	valid_0's auc: 0.714111
[9]	valid_0's auc: 0.720488
[10]	valid_0's auc: 0.732837
[11]	valid_0's auc: 0.737745
[12]	valid_0's auc: 0.748791
[13]	valid_0's auc: 0.758616
[14]	valid_0's auc: 0.763474
[15]	valid_0's auc: 0.766909
[16]	valid_0's auc: 0.767915
[17]	valid_0's auc: 0.76906
[18]	valid_0's auc: 0.777483
[19]	valid_0's auc: 0.778327
[20]	valid_0's auc: 0.780227
[21]	valid_0's auc: 0.782052
[22]	valid_0's auc: 0.780635
[23]	valid_0's auc: 0.779446
[24]	valid_0's auc: 0.783365
[25]	valid_0's auc: 0.784141
[26]	valid_0's auc: 0.788199
[27]	valid_0's auc: 0.790039
[28]	valid_0's auc: 0.795673
[29]	valid_0's auc: 0.795345
[30]	valid_0's auc: 0.797732
[31]	valid_0's auc: 0.79971
[32]	valid_0's auc: 0.798458
[33]	valid_0's auc: 0

[286]	valid_0's auc: 0.865915
[287]	valid_0's auc: 0.866205
[288]	valid_0's auc: 0.866409
[289]	valid_0's auc: 0.86662
[290]	valid_0's auc: 0.866776
[291]	valid_0's auc: 0.866702
[292]	valid_0's auc: 0.866559
[293]	valid_0's auc: 0.866609
[294]	valid_0's auc: 0.866718
[295]	valid_0's auc: 0.866841
[296]	valid_0's auc: 0.867237
[297]	valid_0's auc: 0.867325
[298]	valid_0's auc: 0.86752
[299]	valid_0's auc: 0.867469
[300]	valid_0's auc: 0.867689
[301]	valid_0's auc: 0.86777
[302]	valid_0's auc: 0.868079
[303]	valid_0's auc: 0.868298
[304]	valid_0's auc: 0.868532
[305]	valid_0's auc: 0.868709
[306]	valid_0's auc: 0.868683
[307]	valid_0's auc: 0.868776
[308]	valid_0's auc: 0.869047
[309]	valid_0's auc: 0.869006
[310]	valid_0's auc: 0.869057
[311]	valid_0's auc: 0.869147
[312]	valid_0's auc: 0.869138
[313]	valid_0's auc: 0.869061
[314]	valid_0's auc: 0.869195
[315]	valid_0's auc: 0.86902
[316]	valid_0's auc: 0.868923
[317]	valid_0's auc: 0.868994
[318]	valid_0's auc: 0.869105
[319]	valid_0'

[571]	valid_0's auc: 0.875677
[572]	valid_0's auc: 0.875742
[573]	valid_0's auc: 0.875742
[574]	valid_0's auc: 0.87586
[575]	valid_0's auc: 0.875866
[576]	valid_0's auc: 0.875892
[577]	valid_0's auc: 0.87598
[578]	valid_0's auc: 0.875921
[579]	valid_0's auc: 0.875995
[580]	valid_0's auc: 0.876078
[581]	valid_0's auc: 0.876167
[582]	valid_0's auc: 0.876264
[583]	valid_0's auc: 0.876296
[584]	valid_0's auc: 0.876293
[585]	valid_0's auc: 0.876322
[586]	valid_0's auc: 0.876298
[587]	valid_0's auc: 0.87636
[588]	valid_0's auc: 0.876455
[589]	valid_0's auc: 0.876413
[590]	valid_0's auc: 0.87647
[591]	valid_0's auc: 0.876547
[592]	valid_0's auc: 0.876454
[593]	valid_0's auc: 0.876428
[594]	valid_0's auc: 0.876422
[595]	valid_0's auc: 0.876495
[596]	valid_0's auc: 0.876542
[597]	valid_0's auc: 0.876568
[598]	valid_0's auc: 0.876687
[599]	valid_0's auc: 0.876624
[600]	valid_0's auc: 0.876709
[601]	valid_0's auc: 0.876798
[602]	valid_0's auc: 0.876751
[603]	valid_0's auc: 0.876745
[604]	valid_0'

[854]	valid_0's auc: 0.880625
[855]	valid_0's auc: 0.880642
[856]	valid_0's auc: 0.880647
[857]	valid_0's auc: 0.880707
[858]	valid_0's auc: 0.880717
[859]	valid_0's auc: 0.880723
[860]	valid_0's auc: 0.880744
[861]	valid_0's auc: 0.880716
[862]	valid_0's auc: 0.880734
[863]	valid_0's auc: 0.880749
[864]	valid_0's auc: 0.880768
[865]	valid_0's auc: 0.880789
[866]	valid_0's auc: 0.880837
[867]	valid_0's auc: 0.880853
[868]	valid_0's auc: 0.880841
[869]	valid_0's auc: 0.880897
[870]	valid_0's auc: 0.880925
[871]	valid_0's auc: 0.880957
[872]	valid_0's auc: 0.880979
[873]	valid_0's auc: 0.88101
[874]	valid_0's auc: 0.881074
[875]	valid_0's auc: 0.881096
[876]	valid_0's auc: 0.881135
[877]	valid_0's auc: 0.881151
[878]	valid_0's auc: 0.881195
[879]	valid_0's auc: 0.881223
[880]	valid_0's auc: 0.881268
[881]	valid_0's auc: 0.88129
[882]	valid_0's auc: 0.88133
[883]	valid_0's auc: 0.881377
[884]	valid_0's auc: 0.881413
[885]	valid_0's auc: 0.881417
[886]	valid_0's auc: 0.881447
[887]	valid_0

[1135]	valid_0's auc: 0.884236
[1136]	valid_0's auc: 0.884236
[1137]	valid_0's auc: 0.88424
[1138]	valid_0's auc: 0.884237
[1139]	valid_0's auc: 0.884244
[1140]	valid_0's auc: 0.884277
[1141]	valid_0's auc: 0.884267
[1142]	valid_0's auc: 0.884274
[1143]	valid_0's auc: 0.884313
[1144]	valid_0's auc: 0.88433
[1145]	valid_0's auc: 0.884332
[1146]	valid_0's auc: 0.884321
[1147]	valid_0's auc: 0.8843
[1148]	valid_0's auc: 0.884344
[1149]	valid_0's auc: 0.884337
[1150]	valid_0's auc: 0.884383
[1151]	valid_0's auc: 0.884361
[1152]	valid_0's auc: 0.884382
[1153]	valid_0's auc: 0.884386
[1154]	valid_0's auc: 0.88437
[1155]	valid_0's auc: 0.884348
[1156]	valid_0's auc: 0.884348
[1157]	valid_0's auc: 0.884374
[1158]	valid_0's auc: 0.884347
[1159]	valid_0's auc: 0.884299
[1160]	valid_0's auc: 0.884331
[1161]	valid_0's auc: 0.884326
[1162]	valid_0's auc: 0.884334
[1163]	valid_0's auc: 0.884344
[1164]	valid_0's auc: 0.884363
[1165]	valid_0's auc: 0.884375
[1166]	valid_0's auc: 0.884383
[1167]	valid_

[1420]	valid_0's auc: 0.886966
[1421]	valid_0's auc: 0.886989
[1422]	valid_0's auc: 0.886991
[1423]	valid_0's auc: 0.886996
[1424]	valid_0's auc: 0.887007
[1425]	valid_0's auc: 0.887031
[1426]	valid_0's auc: 0.88703
[1427]	valid_0's auc: 0.887037
[1428]	valid_0's auc: 0.887055
[1429]	valid_0's auc: 0.887068
[1430]	valid_0's auc: 0.887091
[1431]	valid_0's auc: 0.887095
[1432]	valid_0's auc: 0.887084
[1433]	valid_0's auc: 0.887085
[1434]	valid_0's auc: 0.887094
[1435]	valid_0's auc: 0.88713
[1436]	valid_0's auc: 0.887139
[1437]	valid_0's auc: 0.887147
[1438]	valid_0's auc: 0.887161
[1439]	valid_0's auc: 0.887175
[1440]	valid_0's auc: 0.887175
[1441]	valid_0's auc: 0.887206
[1442]	valid_0's auc: 0.887206
[1443]	valid_0's auc: 0.887224
[1444]	valid_0's auc: 0.887228
[1445]	valid_0's auc: 0.887216
[1446]	valid_0's auc: 0.887219
[1447]	valid_0's auc: 0.887226
[1448]	valid_0's auc: 0.887241
[1449]	valid_0's auc: 0.887255
[1450]	valid_0's auc: 0.887255
[1451]	valid_0's auc: 0.887259
[1452]	val

[1705]	valid_0's auc: 0.889333
[1706]	valid_0's auc: 0.889336
[1707]	valid_0's auc: 0.889341
[1708]	valid_0's auc: 0.889333
[1709]	valid_0's auc: 0.889338
[1710]	valid_0's auc: 0.889349
[1711]	valid_0's auc: 0.889368
[1712]	valid_0's auc: 0.889377
[1713]	valid_0's auc: 0.889382
[1714]	valid_0's auc: 0.889409
[1715]	valid_0's auc: 0.889422
[1716]	valid_0's auc: 0.889422
[1717]	valid_0's auc: 0.889434
[1718]	valid_0's auc: 0.889431
[1719]	valid_0's auc: 0.889447
[1720]	valid_0's auc: 0.889474
[1721]	valid_0's auc: 0.889501
[1722]	valid_0's auc: 0.889511
[1723]	valid_0's auc: 0.889532
[1724]	valid_0's auc: 0.889532
[1725]	valid_0's auc: 0.889557
[1726]	valid_0's auc: 0.889579
[1727]	valid_0's auc: 0.889613
[1728]	valid_0's auc: 0.889615
[1729]	valid_0's auc: 0.889633
[1730]	valid_0's auc: 0.889656
[1731]	valid_0's auc: 0.889657
[1732]	valid_0's auc: 0.88967
[1733]	valid_0's auc: 0.889684
[1734]	valid_0's auc: 0.889711
[1735]	valid_0's auc: 0.889751
[1736]	valid_0's auc: 0.889753
[1737]	va

[1990]	valid_0's auc: 0.891395
[1991]	valid_0's auc: 0.891392
[1992]	valid_0's auc: 0.891389
[1993]	valid_0's auc: 0.891398
[1994]	valid_0's auc: 0.891413
[1995]	valid_0's auc: 0.891416
[1996]	valid_0's auc: 0.891437
[1997]	valid_0's auc: 0.891452
[1998]	valid_0's auc: 0.891463
[1999]	valid_0's auc: 0.891469
[2000]	valid_0's auc: 0.891468
[2001]	valid_0's auc: 0.891469
[2002]	valid_0's auc: 0.89146
[2003]	valid_0's auc: 0.891461
[2004]	valid_0's auc: 0.89148
[2005]	valid_0's auc: 0.8915
[2006]	valid_0's auc: 0.891505
[2007]	valid_0's auc: 0.89151
[2008]	valid_0's auc: 0.891523
[2009]	valid_0's auc: 0.891527
[2010]	valid_0's auc: 0.891534
[2011]	valid_0's auc: 0.891545
[2012]	valid_0's auc: 0.891542
[2013]	valid_0's auc: 0.891546
[2014]	valid_0's auc: 0.891549
[2015]	valid_0's auc: 0.89155
[2016]	valid_0's auc: 0.891557
[2017]	valid_0's auc: 0.89156
[2018]	valid_0's auc: 0.891573
[2019]	valid_0's auc: 0.891563
[2020]	valid_0's auc: 0.891559
[2021]	valid_0's auc: 0.891573
[2022]	valid_0'

KeyboardInterrupt: 

In [25]:
# CatBoost
name = 'catboost'
pred = train_predict(name, catboost_model, X, y, T, n_folds=2)

0:	test: 0.5618724	best: 0.5618724 (0)	total: 395ms	remaining: 1h 5m 53s
1:	test: 0.5927932	best: 0.5927932 (1)	total: 801ms	remaining: 1h 6m 42s
2:	test: 0.6033275	best: 0.6033275 (2)	total: 1.31s	remaining: 1h 12m 57s
3:	test: 0.6193035	best: 0.6193035 (3)	total: 1.85s	remaining: 1h 16m 59s
4:	test: 0.6402233	best: 0.6402233 (4)	total: 2.29s	remaining: 1h 16m 13s
5:	test: 0.6504549	best: 0.6504549 (5)	total: 2.73s	remaining: 1h 15m 56s
6:	test: 0.6516974	best: 0.6516974 (6)	total: 3.33s	remaining: 1h 19m 13s
7:	test: 0.6646354	best: 0.6646354 (7)	total: 3.74s	remaining: 1h 17m 48s
8:	test: 0.6694854	best: 0.6694854 (8)	total: 4.15s	remaining: 1h 16m 48s
9:	test: 0.6842202	best: 0.6842202 (9)	total: 4.54s	remaining: 1h 15m 40s
10:	test: 0.6958890	best: 0.6958890 (10)	total: 5.09s	remaining: 1h 17m
11:	test: 0.7026504	best: 0.7026504 (11)	total: 5.53s	remaining: 1h 16m 43s
12:	test: 0.7114027	best: 0.7114027 (12)	total: 5.96s	remaining: 1h 16m 20s
13:	test: 0.7181097	best: 0.7181097 (1

109:	test: 0.8192897	best: 0.8231205 (98)	total: 45.6s	remaining: 1h 8m 19s
110:	test: 0.8198372	best: 0.8231205 (98)	total: 46s	remaining: 1h 8m 14s
111:	test: 0.8205632	best: 0.8231205 (98)	total: 46.4s	remaining: 1h 8m 13s
112:	test: 0.8201467	best: 0.8231205 (98)	total: 46.7s	remaining: 1h 8m 10s
113:	test: 0.8194827	best: 0.8231205 (98)	total: 47.1s	remaining: 1h 8m 7s
114:	test: 0.8201597	best: 0.8231205 (98)	total: 47.5s	remaining: 1h 8m 4s
115:	test: 0.8206368	best: 0.8231205 (98)	total: 47.9s	remaining: 1h 8m 1s
116:	test: 0.8212991	best: 0.8231205 (98)	total: 48.2s	remaining: 1h 7m 55s
117:	test: 0.8210623	best: 0.8231205 (98)	total: 48.6s	remaining: 1h 7m 49s
118:	test: 0.8212794	best: 0.8231205 (98)	total: 49s	remaining: 1h 7m 45s
119:	test: 0.8218355	best: 0.8231205 (98)	total: 49.3s	remaining: 1h 7m 41s
120:	test: 0.8225063	best: 0.8231205 (98)	total: 49.7s	remaining: 1h 7m 36s
121:	test: 0.8225782	best: 0.8231205 (98)	total: 50s	remaining: 1h 7m 31s
122:	test: 0.8225564	

216:	test: 0.8301272	best: 0.8301731 (215)	total: 1m 25s	remaining: 1h 4m 21s
217:	test: 0.8302337	best: 0.8302337 (217)	total: 1m 26s	remaining: 1h 4m 20s
218:	test: 0.8304191	best: 0.8304191 (218)	total: 1m 26s	remaining: 1h 4m 21s
219:	test: 0.8303364	best: 0.8304191 (218)	total: 1m 26s	remaining: 1h 4m 21s
220:	test: 0.8301821	best: 0.8304191 (218)	total: 1m 27s	remaining: 1h 4m 23s
221:	test: 0.8301455	best: 0.8304191 (218)	total: 1m 27s	remaining: 1h 4m 22s
222:	test: 0.8302123	best: 0.8304191 (218)	total: 1m 28s	remaining: 1h 4m 20s
223:	test: 0.8302503	best: 0.8304191 (218)	total: 1m 28s	remaining: 1h 4m 21s
224:	test: 0.8302595	best: 0.8304191 (218)	total: 1m 28s	remaining: 1h 4m 20s
225:	test: 0.8303301	best: 0.8304191 (218)	total: 1m 29s	remaining: 1h 4m 18s
226:	test: 0.8303543	best: 0.8304191 (218)	total: 1m 29s	remaining: 1h 4m 16s
227:	test: 0.8305990	best: 0.8305990 (227)	total: 1m 29s	remaining: 1h 4m 14s
228:	test: 0.8308486	best: 0.8308486 (228)	total: 1m 30s	remaini

322:	test: 0.8397858	best: 0.8397858 (322)	total: 2m 5s	remaining: 1h 2m 47s
323:	test: 0.8397518	best: 0.8397858 (322)	total: 2m 6s	remaining: 1h 2m 49s
324:	test: 0.8397119	best: 0.8397858 (322)	total: 2m 6s	remaining: 1h 2m 48s
325:	test: 0.8397303	best: 0.8397858 (322)	total: 2m 6s	remaining: 1h 2m 47s
326:	test: 0.8397673	best: 0.8397858 (322)	total: 2m 7s	remaining: 1h 2m 47s
327:	test: 0.8398722	best: 0.8398722 (327)	total: 2m 7s	remaining: 1h 2m 47s
328:	test: 0.8400416	best: 0.8400416 (328)	total: 2m 8s	remaining: 1h 2m 47s
329:	test: 0.8402580	best: 0.8402580 (329)	total: 2m 8s	remaining: 1h 2m 47s
330:	test: 0.8401941	best: 0.8402580 (329)	total: 2m 8s	remaining: 1h 2m 45s
331:	test: 0.8403234	best: 0.8403234 (331)	total: 2m 9s	remaining: 1h 2m 46s
332:	test: 0.8402943	best: 0.8403234 (331)	total: 2m 9s	remaining: 1h 2m 47s
333:	test: 0.8403351	best: 0.8403351 (333)	total: 2m 10s	remaining: 1h 2m 47s
334:	test: 0.8403012	best: 0.8403351 (333)	total: 2m 10s	remaining: 1h 2m 4

428:	test: 0.8472945	best: 0.8472945 (428)	total: 2m 47s	remaining: 1h 2m 12s
429:	test: 0.8473048	best: 0.8473048 (429)	total: 2m 47s	remaining: 1h 2m 13s
430:	test: 0.8473782	best: 0.8473782 (430)	total: 2m 48s	remaining: 1h 2m 12s
431:	test: 0.8474617	best: 0.8474617 (431)	total: 2m 48s	remaining: 1h 2m 12s
432:	test: 0.8474723	best: 0.8474723 (432)	total: 2m 48s	remaining: 1h 2m 12s
433:	test: 0.8475538	best: 0.8475538 (433)	total: 2m 49s	remaining: 1h 2m 11s
434:	test: 0.8475256	best: 0.8475538 (433)	total: 2m 49s	remaining: 1h 2m 11s
435:	test: 0.8475790	best: 0.8475790 (435)	total: 2m 50s	remaining: 1h 2m 9s
436:	test: 0.8476284	best: 0.8476284 (436)	total: 2m 50s	remaining: 1h 2m 8s
437:	test: 0.8477310	best: 0.8477310 (437)	total: 2m 50s	remaining: 1h 2m 7s
438:	test: 0.8477869	best: 0.8477869 (438)	total: 2m 51s	remaining: 1h 2m 6s
439:	test: 0.8478840	best: 0.8478840 (439)	total: 2m 51s	remaining: 1h 2m 5s
440:	test: 0.8479618	best: 0.8479618 (440)	total: 2m 51s	remaining: 1

...

9506:	test: 0.8926266	best: 0.8926286 (9442)	total: 56m 6s	remaining: 2m 54s
9507:	test: 0.8926257	best: 0.8926286 (9442)	total: 56m 7s	remaining: 2m 54s
9508:	test: 0.8926242	best: 0.8926286 (9442)	total: 56m 7s	remaining: 2m 53s
9509:	test: 0.8926278	best: 0.8926286 (9442)	total: 56m 7s	remaining: 2m 53s
9510:	test: 0.8926261	best: 0.8926286 (9442)	total: 56m 8s	remaining: 2m 53s
9511:	test: 0.8926267	best: 0.8926286 (9442)	total: 56m 8s	remaining: 2m 52s
9512:	test: 0.8926256	best: 0.8926286 (9442)	total: 56m 9s	remaining: 2m 52s
9513:	test: 0.8926263	best: 0.8926286 (9442)	total: 56m 9s	remaining: 2m 52s
9514:	test: 0.8926285	best: 0.8926286 (9442)	total: 56m 9s	remaining: 2m 51s
9515:	test: 0.8926289	best: 0.8926289 (9515)	total: 56m 10s	remaining: 2m 51s
9516:	test: 0.8926308	best: 0.8926308 (9516)	total: 56m 10s	remaining: 2m 51s
9517:	test: 0.8926278	best: 0.8926308 (9516)	total: 56m 10s	remaining: 2m 50s
9518:	test: 0.8926277	best: 0.8926308 (9516)	total: 56m 11s	remaining: 2m

9612:	test: 0.8926473	best: 0.8926479 (9609)	total: 56m 44s	remaining: 2m 17s
9613:	test: 0.8926470	best: 0.8926479 (9609)	total: 56m 44s	remaining: 2m 16s
9614:	test: 0.8926446	best: 0.8926479 (9609)	total: 56m 44s	remaining: 2m 16s
9615:	test: 0.8926426	best: 0.8926479 (9609)	total: 56m 45s	remaining: 2m 15s
9616:	test: 0.8926430	best: 0.8926479 (9609)	total: 56m 45s	remaining: 2m 15s
9617:	test: 0.8926423	best: 0.8926479 (9609)	total: 56m 45s	remaining: 2m 15s
9618:	test: 0.8926412	best: 0.8926479 (9609)	total: 56m 46s	remaining: 2m 14s
9619:	test: 0.8926413	best: 0.8926479 (9609)	total: 56m 46s	remaining: 2m 14s
9620:	test: 0.8926422	best: 0.8926479 (9609)	total: 56m 46s	remaining: 2m 14s
9621:	test: 0.8926434	best: 0.8926479 (9609)	total: 56m 47s	remaining: 2m 13s
9622:	test: 0.8926438	best: 0.8926479 (9609)	total: 56m 47s	remaining: 2m 13s
9623:	test: 0.8926423	best: 0.8926479 (9609)	total: 56m 47s	remaining: 2m 13s
9624:	test: 0.8926405	best: 0.8926479 (9609)	total: 56m 48s	rema

9719:	test: 0.8926551	best: 0.8926657 (9710)	total: 57m 21s	remaining: 1m 39s
9720:	test: 0.8926531	best: 0.8926657 (9710)	total: 57m 21s	remaining: 1m 38s
9721:	test: 0.8926510	best: 0.8926657 (9710)	total: 57m 21s	remaining: 1m 38s
9722:	test: 0.8926507	best: 0.8926657 (9710)	total: 57m 22s	remaining: 1m 38s
9723:	test: 0.8926504	best: 0.8926657 (9710)	total: 57m 22s	remaining: 1m 37s
9724:	test: 0.8926526	best: 0.8926657 (9710)	total: 57m 22s	remaining: 1m 37s
9725:	test: 0.8926551	best: 0.8926657 (9710)	total: 57m 23s	remaining: 1m 37s
9726:	test: 0.8926543	best: 0.8926657 (9710)	total: 57m 23s	remaining: 1m 36s
9727:	test: 0.8926540	best: 0.8926657 (9710)	total: 57m 23s	remaining: 1m 36s
9728:	test: 0.8926539	best: 0.8926657 (9710)	total: 57m 24s	remaining: 1m 35s
9729:	test: 0.8926553	best: 0.8926657 (9710)	total: 57m 24s	remaining: 1m 35s
9730:	test: 0.8926550	best: 0.8926657 (9710)	total: 57m 24s	remaining: 1m 35s
9731:	test: 0.8926537	best: 0.8926657 (9710)	total: 57m 25s	rema

9825:	test: 0.8926563	best: 0.8926657 (9710)	total: 57m 57s	remaining: 1m 1s
9826:	test: 0.8926557	best: 0.8926657 (9710)	total: 57m 58s	remaining: 1m 1s
9827:	test: 0.8926553	best: 0.8926657 (9710)	total: 57m 58s	remaining: 1m
9828:	test: 0.8926555	best: 0.8926657 (9710)	total: 57m 58s	remaining: 1m
9829:	test: 0.8926565	best: 0.8926657 (9710)	total: 57m 59s	remaining: 1m
9830:	test: 0.8926577	best: 0.8926657 (9710)	total: 57m 59s	remaining: 59.8s
9831:	test: 0.8926579	best: 0.8926657 (9710)	total: 57m 59s	remaining: 59.5s
9832:	test: 0.8926577	best: 0.8926657 (9710)	total: 58m	remaining: 59.1s
9833:	test: 0.8926575	best: 0.8926657 (9710)	total: 58m	remaining: 58.8s
9834:	test: 0.8926573	best: 0.8926657 (9710)	total: 58m	remaining: 58.4s
9835:	test: 0.8926584	best: 0.8926657 (9710)	total: 58m 1s	remaining: 58s
9836:	test: 0.8926571	best: 0.8926657 (9710)	total: 58m 1s	remaining: 57.7s
9837:	test: 0.8926554	best: 0.8926657 (9710)	total: 58m 2s	remaining: 57.3s
9838:	test: 0.8926580	bes

9933:	test: 0.8926485	best: 0.8926657 (9710)	total: 58m 35s	remaining: 23.4s
9934:	test: 0.8926485	best: 0.8926657 (9710)	total: 58m 35s	remaining: 23s
9935:	test: 0.8926507	best: 0.8926657 (9710)	total: 58m 36s	remaining: 22.6s
9936:	test: 0.8926518	best: 0.8926657 (9710)	total: 58m 36s	remaining: 22.3s
9937:	test: 0.8926519	best: 0.8926657 (9710)	total: 58m 36s	remaining: 21.9s
9938:	test: 0.8926548	best: 0.8926657 (9710)	total: 58m 37s	remaining: 21.6s
9939:	test: 0.8926554	best: 0.8926657 (9710)	total: 58m 37s	remaining: 21.2s
9940:	test: 0.8926525	best: 0.8926657 (9710)	total: 58m 37s	remaining: 20.9s
9941:	test: 0.8926533	best: 0.8926657 (9710)	total: 58m 38s	remaining: 20.5s
9942:	test: 0.8926493	best: 0.8926657 (9710)	total: 58m 38s	remaining: 20.2s
9943:	test: 0.8926470	best: 0.8926657 (9710)	total: 58m 38s	remaining: 19.8s
9944:	test: 0.8926505	best: 0.8926657 (9710)	total: 58m 39s	remaining: 19.5s
9945:	test: 0.8926501	best: 0.8926657 (9710)	total: 58m 39s	remaining: 19.1s
9

In [ ]:
# Ensemble
name = 'ensemble'
base_models = [xgboost_model, lightgbm_model, catboost_model]
model_names = ['xgboost', 'lightgbm', 'catboost']
ensemble = Ensemble(lightgbm_stacker_model, base_models, model_names, n_folds=5)
pred = ensemble.fit_predict(X, y, T)

In [26]:
# submission file
sub_df = pd.DataFrame({'ID_code': test_df['ID_code'].values, 'target': pred})
sub_df.to_csv('../submit/{}_submission.csv'.format(name), index=False)